In [17]:
# Clone PyFM
!git clone https://github.com/RobinMagnet/pyFM.git pyfm_repo
!mv pyfm_repo/pyFM .
!rm -rf pyfm_repo

Cloning into 'pyfm_repo'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 512 (delta 59), reused 56 (delta 50), pack-reused 436
Receiving objects: 100% (512/512), 942.38 KiB | 10.83 MiB/s, done.
Resolving deltas: 100% (324/324), done.
mv: cannot move 'pyfm_repo/pyFM' to './pyFM': Directory not empty



# 1 - Imports and defining functions

In [2]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

def plot_correspondence_lines(mesh1, mesh2, mesh1_points, mesh2_points):
    def plot_mesh_and_lines(mesh, points):
        p = mp.plot(mesh1_points, None, c=visu(points))
        p.add_mesh(mesh.vertices, mesh.faces, c=visu(mesh.vertices))
        p.add_lines(mesh1_points, mesh2_points)
    plot_mesh_and_lines(mesh1, mesh2_points)
    plot_mesh_and_lines(mesh2, mesh1_points)

Select the shape

In [48]:
toy = ['blue', 'yellow'][1]
num = 2

if toy == 'yellow':
    prefix = 'yellow_'
    dir_name = 'YellowToy01'
else:
    prefix = ''
    dir_name = 'BlueToy02'

# 2 - Computing the functional map

**Loading data**

In [40]:
mesh2 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_1_70000.obj')
mesh1 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_{num}_70000.obj')
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 36821 vertices, 70000 faces
Mesh 2 : 35002 vertices, 70000 faces


In [7]:
"""from scipy.spatial import distance
import numpy as np
rand_idx=np.random.permutation(mesh1.vertlist.shape[0])[:20]
print(rand_idx)
euc_dist=distance.cdist(mesh1.vertlist[rand_idx],mesh2.vertlist)"""

[  809 10714  2534 23241  3511  9163 23014 13152 21086  9417 35548 29794
 23911 26182  4867 10375 18480 18081 35265  7716]


In [8]:
#temp=np.argmin(euc_dist, axis=1)

In [23]:
# Manually change the landmark file name
"""with open("data/yellow_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")

with open("data/yellow_toy_1-"+str(num)+"_landmarks_non_and_deformed.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(temp[i])+" "+str(rand_idx[i])+"\n")
"""

'with open("data/yellow_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt", \'w+\') as fp:\n    \n    for i in range(temp.shape[0]):\n        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")\n\nwith open("data/yellow_toy_1-"+str(num)+"_landmarks_non_and_deformed.txt", \'w+\') as fp:\n    \n    for i in range(temp.shape[0]):\n        fp.write(str(temp[i])+" "+str(rand_idx[i])+"\n")\n'

**Computing descriptors**

In [41]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt(f'data/{toy}_toy_1-{num}_landmarks_deformed_and_non.txt',dtype=int),  # loading 10 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 13.16 s
Computing 200 eigenvectors
	Done in 13.48 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept


**Fitting the model**

$\renewcommand{\RR}{\mathbb{R}}$
$\renewcommand{\Ss}{\mathcal{S}}$
$\renewcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\renewcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [42]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 1.4e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 17, nit : 15, warnflag : 0
	Done in 0.74 seconds


**Visualizing the associated point to point map**

In [43]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 3 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [27]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

**Zoomout**

In [44]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(50, 50)


In [45]:
import trimesh
from scipy.spatial import distance
p2p_21_zo_35000=p2p_21_zo[:35000]
print(mesh1.vertlist[p2p_21_zo_35000].shape,mesh2.vertlist[:35000].shape)
np.save(f'Non-deformed-toDeformed Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.npy',mesh1.vertlist[p2p_21_zo_35000])
np.save(f'Non-deformed-toDeformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.npy',mesh2.vertlist[:35000])
cloud_1 = trimesh.PointCloud(mesh1.vertlist[p2p_21_zo_35000])
cloud_1.export(f'Non-deformed-toDeformed Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.ply')
cloud_2 = trimesh.PointCloud(mesh2.vertlist[:35000])
cloud_2.export(f'Non-deformed-toDeformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.ply')
euc_dist=distance.cdist(cloud_2.vertices,cloud_1.vertices)
min_idx=np.argmin(euc_dist, axis=1)
min_vals=np.min(euc_dist,axis=1)
deformation_dist=np.max(min_vals)
deformation_idx=np.argmax(min_vals)
f=open(f"Non-deformed-toDeformed Correspondences/{dir_name}/{toy}_deformation_1_{num}_35000.txt","w")
f.write("1 "+ str(num)+"\n")
f.write(str(cloud_2.vertices[deformation_idx])+" "+str(cloud_1.vertices[min_idx[deformation_idx]])+"\n")
f.write(str(deformation_idx)+" "+str(min_idx[deformation_idx])+"\n")
f.write(str(cloud_1.vertices[min_idx[deformation_idx]]-cloud_2.vertices[deformation_idx])+"\n")
f.write(str(deformation_dist))
f.close()

(35000, 3) (35000, 3)


In [47]:
random_indexes=np.random.permutation(p2p_21_zo.shape[0])[:2048]
p2p_21_zo_2048=p2p_21_zo[random_indexes]
np.save(f'Non-deformed-toDeformed Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_2048.npy',mesh1.vertlist[p2p_21_zo_2048])
np.save(f'Non-deformed-toDeformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_2048.npy',mesh2.vertlist[random_indexes])
cloud_1_2048 = trimesh.PointCloud(mesh1.vertlist[p2p_21_zo_2048])
cloud_1_2048.export(f'Non-deformed-toDeformed Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_2048.ply')
cloud_2_2048 = trimesh.PointCloud(mesh2.vertlist[random_indexes])
cloud_2_2048.export(f'Non-deformed-toDeformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_2048.ply')


b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 2048\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nend_header\nK\xb0\xb8</k"=\xe77\x1c>\x00\x00\x00\x00\xb2L\x9f=3\x16\r\xbd\xde\xab6>\x00\x00\x00\x00\xa8\x1b\xa8<\x7f\xa3]\xbdaO;>\x00\x00\x00\x00m\xa9\x03\xbb\xb8Y\xbc\xba\x02\xd9\x1b>\x00\x00\x00\x000\xf5\x93=\x9e\xd2\xc1<\x88\x81\x1e>\x00\x00\x00\x00#j"<\xcc\x9b\xc3\xbcc\x0c<>\x00\x00\x00\x00\x1e\x17U\xbc\xa3?\xb4<A\x0e:>\x00\x00\x00\x00i\xe3\x08\xbd\x1e\x87A\xbc\xa6\xd2/>\x00\x00\x00\x00\x96\x96\x11<\x0e\xbdE\xbdJ\xed5>\x00\x00\x00\x00\xe6\x05X<\xc5t!\xbdR\xf2:>\x00\x00\x00\x00\x04\xaek=?\xab\x0c=t\x99:>\x00\x00\x00\x00<\xa1W\xbc\xb7\xb2\xc4<\x94\xf8\x1c>\x00\x00\x00\x00\xcb\x10G<\x9f\x00J=\xa2\x9c8>\x00\x00\x00\x00 c\x8e=\xd5$x=\xaf\x06(>\x00\x00\x00\x00\xad\xfc2\xbd\xb2Ji<\x1cx%>\x00\x00\x00\x00be4=\x9c5\x98=\x91\x99+>\x00\x00\x00\x00y:\xb7=\

In [53]:
cloud_1_2048=np.load(f'Non-deformed-toDeformed Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.npy')
cloud_2_2048=np.load(f'Non-deformed-toDeformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.npy')

distances=np.linalg.norm(cloud_2_2048-cloud_1_2048,ord=2, axis=1)
max_val=np.max(distances,axis=0)
max_idx=np.argmax(distances,axis=0)
f=open(f"Non-deformed-toDeformed Correspondences/{dir_name}/{toy}_deformation_1_{num}_35000.txt","w")
f.write("1 "+ str(num)+"\n")
f.write(str(max_idx)+"\n")
f.write(str(max_val))
f.close()

In [54]:
distances

array([0.00391676, 0.00295062, 0.01837294, ..., 0.04314453, 0.00443057,
       0.00987812])

In [10]:
!pip install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.6/681.6 kB 6.0 MB/s eta 0:00:0000:0100:01


### Finding bijective Correspondences

In [31]:
a=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/deformed_{num}_correspondences_icp.npy')
b=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/deformed_{num}_correspondences_icp.npy')

c=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_icp.npy')
d=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_icp.npy')

deformed_num_icp_deformed_to_non_blue=a.tolist()
deformed_num_icp_non_to_deformed_blue=b.tolist()
nondeformed_num_icp_deformed_to_non_blue=c.tolist()
nondeformed_num_icp_non_to_deformed_blue=d.tolist()
list_bijective_deformed_icp_blue=[]
list_bijective_nondeformed_icp_blue=[]

for i,row in enumerate(deformed_num_icp_deformed_to_non_blue):
    try:
        id=deformed_num_icp_non_to_deformed_blue.index(row)
        if nondeformed_num_icp_deformed_to_non_blue[i]==nondeformed_num_icp_non_to_deformed_blue[id]:
            list_bijective_deformed_icp_blue.append(row)
            list_bijective_nondeformed_icp_blue.append(nondeformed_num_icp_deformed_to_non_blue[i])
    except:
        continue
print(len(list_bijective_deformed_icp_blue),len(list_bijective_nondeformed_icp_blue))
np.save(f'Bijective Correspondences/{dir_name}/deformed_1-{num}_bijective_icp.npy',np.array(list_bijective_deformed_icp_blue))
"""for row in nondeformed_num_icp_deformed_to_non_blue:
    if row in nondeformed_num_icp_non_to_deformed_blue:
        list_bijective_nondeformed_icp_blue.append(row)"""
np.save(f'Bijective Correspondences/{dir_name}/nondeformed_1-{num}_bijective_icp.npy',np.array(list_bijective_nondeformed_icp_blue))

e=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')
f=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')

g=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')
h=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')

deformed_num_zoomed_deformed_to_non_blue=a.tolist()
deformed_num_zoomed_non_to_deformed_blue=b.tolist()
nondeformed_num_zoomed_deformed_to_non_blue=c.tolist()
nondeformed_num_zoomed_non_to_deformed_blue=d.tolist()
list_bijective_deformed_zoomed_blue=[]
list_bijective_nondeformed_zoomed_blue=[]

for i,row in enumerate(deformed_num_zoomed_deformed_to_non_blue):
    try:
        id=deformed_num_zoomed_non_to_deformed_blue.index(row)
        if nondeformed_num_zoomed_deformed_to_non_blue[i]==nondeformed_num_zoomed_non_to_deformed_blue[id]:
            list_bijective_deformed_zoomed_blue.append(row)
            list_bijective_nondeformed_zoomed_blue.append(nondeformed_num_zoomed_deformed_to_non_blue[i])
            #print(id,i,row,nondeformed_num_zoomed_deformed_to_non_blue[i],nondeformed_num_zoomed_non_to_deformed_blue[id])
    except:
        continue
print(len(list_bijective_deformed_zoomed_blue),len(list_bijective_nondeformed_zoomed_blue))
np.save(f'Bijective Correspondences/{dir_name}/deformed_1-{num}_bijective_zoomed.npy',np.array(list_bijective_deformed_zoomed_blue))
"""for row in nondeformed_num_zoomed_deformed_to_non_blue:
    if row in nondeformed_num_zoomed_non_to_deformed_blue:
        list_bijective_nondeformed_zoomed_blue.append(row)"""
np.save(f'Bijective Correspondences/{dir_name}/nondeformed_1-{num}_bijective_zoomed.npy',np.array(list_bijective_nondeformed_zoomed_blue))


29 29
29 29


In [33]:
plot_correspondence_lines(mesh1, mesh2, nondeformed, deformed)
print(deformed.shape)
print(nondeformed.shape)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.046310…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.046310…

(29, 3)
(29, 3)


In [34]:
print(mesh1.vertices.shape)
print(d.shape)
plot_correspondence_lines(mesh1, mesh2, mesh1.vertices, d)

(35002, 3)
(35002, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.054015…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.054015…